# Problem 4

## 1) Explain the difference between episodic and continuous tasks? Given an example of each.

The difference between episodic and continuous tasks is that in the case of continuous tasks, the task might go on forever without terminating, while episodic tasks will have a terminal state that will be reached in a finite amount of time. An episodic task could be a car racing game, where one race could be considered a single episode. In the case of continuous, one example could be a robot who's task is to be a personal assistant or a trading bot. 

## 2) What do the terms exploration and exploitation mean in RL ? Why do the actors employ ϵ-greedy policy for selecting actions at each step? Should ϵ remain fixed or follow a schedule during Deep RL training ? How does the value of ϵ help balance exploration and exploitation during training.

Exploration refers to the system's attempts to discover new features about the world by selecting actions that are not optimal (or are not part of the current policy). On the other hand, exploitation simply refers to using the things that the system already knows to get the best possible result or in other words, to maximize the reward.

Actors employ $\epsilon$-greedy policy because they want to allow some exploration while mainly maintaining the greedy policy. According to the paper, the value of $\epsilon$ starts at 1, and is annealed linearly to a small value of 0.1 over over the ﬁrst million frames, and ﬁxed at 0.1 thereafter.

If $\epsilon = 0$, then the policy becomes the greedy policy, and on the other hand, if $\epsilon = 1$, then the actor will always explore. This means that by changing the value of $\epsilon$, we will change how much exploration and exploitation we do. As we increase $\epsilon$, the amount of exploitation decreases, while if we decrease $\epsilon$, then the the amount of exploration also decreases. It seems to be a good idea to keep this value low to allow for some exploration while maximizing the reward.  



## 3) How is the Deep Q-Learning algorithm different from Q-learning ? You will follow the steps of Deep Q-Learning algorithm in Mnih et al. (2013) page 5, and explain each step in your own words.

The main difference between Deep Q-Learning and regular Q-learning is the use of neural networks to map input states to actions and q-values. I also think that the use of experience replay is also an important feature of deep Q-learning. 

Below is line by line explanation of the algorithm. 

![](images/deepQlearning.png)

In the first line, we initialize the experience replay buffer $D$. When then treat $Q$ as a neural network and initialize it with random weights. 

In the third line we start going episode by episode, and the first thing we do is preprocess the sequence of frames and build the correct input. 

Then in the second for loop, we go through each step and determine which action we are going to take next (with $\epsilon$-greedy policy as explained in the previous questions). We then observe the reward and the new state.

We then set the next state $s_{t+1} = s_t, a_t, x_{t+1}$ and then we store this transition in the replay buffer. 

Finally, we sample randomly from $D$ to get the input for the deep neural network, and to get the value of y to be able to train it, we first determine if this is the end of the episode or not. If it is the end, then y is simply the reward, else, y is the reward plus the reward that we will get in the future. To complete this iteration, we train the network using gradient descent. 

## 4) What is the benefit of having a target Q-network ?

The target Q-network is important because it stabilizes learning. According to the paper, "The target Q-network is used to generate target Q values for the DQN loss term and the replay memory that the agent uses to sample random transitions for training the Q-network". As I understand it, the Q-network is updated every step, this could introduce a lot of noise so we copy the weights from the Q-network into the target Q-network to avoid this. This also keeps the target function from changing too quickly.  

## 5) How does experience replay help in efficient Q-learning ?

According to the paper by Schaul et al, "Experience replay liberates online learning agents from processing transitions in the exact order they are experienced."

The experience replay is an important part of the algorithm because it maintains the Q-value for state action pairs, which allows us to logically separate the acting and learning stages. In the acting stage ge gather data from our actions, while in the learning stage, we take random samples from the data that we have seen (obviously because of memory constraints, we can't keep everything so keep a buffer instead). This means that we are using the experience that we gather more efficiently because we could use one training example multiple times. Also since the distribution of the training data is changing constantly because the agent's actions affect the environment, having the experience replay allows us to (at least in practice) make the assumption that the training data is independent and identically distributed while we train the neural network. 

Also, from the paper, they say: "Experience replay may also help to prevent overﬁtting by allowing the agent to learn from data generated by previous versions of the policy."

## 6) What is prioritized experience replay and how is priority of a sample calculated?

Prioritized experience replay is a technique where instead of sampling uniformly from the experience replay, we sample the most important samples (the samples that will help learning converge faster) more frequently than others. In the paper by Horgan et al, they implemented the algorithm for proportional prioritization described in Schaul et al. (2016). This priority is expressed in terms of expected learning progress, as measured by the magnitude of their temporal-difference (TD) error.

"the magnitude of a transition’s TD error δ, indicates how ‘surprising’ or unexpected the transition is: specifically, how far the value is from its next-step bootstrap estimate"

We can calculate $\delta$ as 

$\delta = r_t + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)$

Now the probability of sampling sample $i$ is

$P(i) = \frac{p_i^{\alpha}}{\sum_k p_k^{\alpha}}$

here $p_i$ is the probability of transition $i$, $\alpha$ determines how much prioritization is used, $\alpha = 0$ is the uniform case. 

To make up for the bias that sampling using TD has introduced, during training, in every step we multiply the gradient by the importance sampling weights of the experiences, which is computed as: 

$w_i = (\frac{1}{N} \cdot \frac{1}{P(i)})^{\beta}$

where $\beta$ is the annealing rate for bias correction, and $N$ is the batch size. 


## 7) Compare and contrast GORILA (General Reinforcement Learning Architecture) and Ape-X architecture. Provide three similarities and three differences.

Differences: 
* ApeX uses a prioritized replay buffer while Gorila doesn't. 
* ApeX shares experiences instead of gradients, while Gorila uses a parameter server to update the gradients. 
* In ApeX, each actor executes a different policy, while in Gorila we don't necessarily start with different policies for each actor. 

Similarities: 
* They both decompose the problem into stages that can run concurrently, the acting stage and the training stage.
* They both store the experiences in a server that can be reached by all the learners.
* Both architectures use a target network.  

## 8) Why the performance improves with number of actors in Ape-X?

The authors of the paper hypothesize that this happens because in some cases the policy discovered is a local optimum in the parameter space, but no a global one due to insufficient exploration. If we use many actors, each with a different amount of exploration, they will discover promising new courses of action, and with priority experience replay, if this happens, the algorithm will use this samples more often during training, which in turns helps the performance. 